In [1]:
#This notebook is split into two parts.

#PART 1 uploads market data for USD interest rate swaps (can easily change to overnight index swaps)
#from Bloomberg's API, using Brian Smith's beautiful TIA wrapper (https://github.com/bpsmith/tia),
#which converts the data into a panda dataframe.
#Data feed can replaced with a different provider, or skipped with instruments keyed manually in PART 2.

#If you have no data source, you can go directly to PART 2, keying data manually.  

##### PART 1 of 2 ######

#for BLPAPI to work, copy blpapi folder into your Python Anaconda directory.
#mine is  C:\Users\cp85vc\AppData\Local\Continuum\Anaconda3\lib\site-packages\ 

#Make SURE it includes the blpapi3_64.dll from Bloomberg's installer directory: C:\blp\API\APIv3\C++API\v3.8.13.1\lib
#then go to Start, type PATH
#add as VariableValue:
#C:\Users\cp85vc\AppData\Local\Continuum\Anaconda3\Lib\site-packages\blpapi '''

import blpapi #to retrieve real time market data from Bloomberg
from tia.bbg import LocalTerminal  #to have the data from Bloomberg in a presentable, dataframe table fashion
import pandas as pd #to access dataframes
import numpy as np #to interpolate missing market data points, like 16year or 23year swaps
import matplotlib.pylab as plt # to plot graphs

import datetime  
today = pd.datetime.today()

In [2]:
#like with BLPAPI, download the TIA package from GitHub and paste in the directory:
#C:\Users\cp85vc\AppData\Local\Continuum\Anaconda3\Lib\site-packages\tia

#There was a NameError when calling 'resp', 'basestring' not defined
#from C:\Users\cp85vc\AppData\Local\Continuum\Anaconda3\lib\site-packages\tia\bbg\v3api.py 

#So had to change all them to 'str', close-re-open and then it worked :)

#Examples :  http://nbviewer.jupyter.org/github/bpsmith/tia/blob/master/examples/v3api.ipynb '''

In [3]:
#import USD IRS swaps from 1y to 30yrs and save into a panda dataframe
df = LocalTerminal.get_historical(['USSA1 Curncy','USSA2 Curncy', 'USSA3 Curncy',\
                                      'USSA4 Curncy', 'USSA5 Curncy', 'USSA6 Curncy', \
                                      'USSA7 Curncy', 'USSA8 Curncy', 'USSA9 Curncy', \
                                      'USSA10 Curncy','USSA11 Curncy', 'USSA12 Curncy',\
                                     'USSA15 Curncy', 'USSA20 Curncy',\
                                    'USSA25 Curncy', 'USSA30 Curncy'],'PX_LAST', \
                                    start= today, end= today, period = 'DAILY')
USSA = df.as_frame().tail()
USSA #this is a multi index dataframe with: 'USSA_year Currency' and 'PX_LAST' as indeces.  

,USSA2 Curncy,USSA12 Curncy,USSA25 Curncy,USSA6 Curncy,USSA1 Curncy,USSA5 Curncy,USSA3 Curncy,USSA30 Curncy,USSA9 Curncy,USSA7 Curncy,USSA11 Curncy,USSA4 Curncy,USSA8 Curncy,USSA15 Curncy,USSA10 Curncy,USSA20 Curncy
,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST
date,,,,,,,,,,,,,,,,
2017-11-07,1.8231,2.3548,2.5092,2.112,1.65,2.057,1.9226,2.5134,2.2505,2.1673,2.3285,1.9955,2.2092,2.4281,2.2895,2.4863


In [4]:
#remove the "PX_LAST" index
USSA.columns = USSA.columns.get_level_values(0)
USSA

,USSA2 Curncy,USSA12 Curncy,USSA25 Curncy,USSA6 Curncy,USSA1 Curncy,USSA5 Curncy,USSA3 Curncy,USSA30 Curncy,USSA9 Curncy,USSA7 Curncy,USSA11 Curncy,USSA4 Curncy,USSA8 Curncy,USSA15 Curncy,USSA10 Curncy,USSA20 Curncy
date,,,,,,,,,,,,,,,,
2017-11-07,1.8231,2.3548,2.5092,2.112,1.65,2.057,1.9226,2.5134,2.2505,2.1673,2.3285,1.9955,2.2092,2.4281,2.2895,2.4863


In [5]:
#the columns are in a weird order. So we relabel them dropping the "Curncy" suffix, which is 6 letters, 
#so that we can sort them
USSA = USSA.rename(columns = lambda x : str(x)[:-6]) 
#this iterates through the columns and apply some function.
#In this case, the function converts something into a string and takes everything up until the last 6 characters, ie the "Curncy"
USSA

,USSA2,USSA12,USSA25,USSA6,USSA1,USSA5,USSA3,USSA30,USSA9,USSA7,USSA11,USSA4,USSA8,USSA15,USSA10,USSA20
date,,,,,,,,,,,,,,,,
2017-11-07,1.8231,2.3548,2.5092,2.112,1.65,2.057,1.9226,2.5134,2.2505,2.1673,2.3285,1.9955,2.2092,2.4281,2.2895,2.4863


In [6]:
#..sort them and  divide by 100 as BBG data is 1.605%
USSA = USSA.reindex_axis(sorted(USSA.columns, key = lambda x: float(x[4: ])),axis = 1).divide(100, axis = 'columns')
USSA

,USSA1,USSA2,USSA3,USSA4,USSA5,USSA6,USSA7,USSA8,USSA9,USSA10,USSA11,USSA12,USSA15,USSA20,USSA25,USSA30
date,,,,,,,,,,,,,,,,
2017-11-07,0.0165,0.018231,0.019226,0.019955,0.02057,0.02112,0.021673,0.022092,0.022505,0.022895,0.023285,0.023548,0.024281,0.024863,0.025092,0.025134


In [7]:
#want to graph yield curve?
transp = USSA.transpose()  #first transpose, so that 1x30 matrix becomes 30rows (one per year) x 1 column (for values)
transp.plot(color = 'blue')

plt.title('USSA Curve')
plt.xlabel('Years')
plt.ylabel('US Swaps')

#set limits
plt.xlim(0,16)
plt.ylim(0, 0.04)

plt.grid('on')
plt.show()

In [8]:
#sanity check
USSA.iloc[0,0], USSA.iloc[0, 1], USSA.iloc[0, 2], USSA.iloc[0, 3], USSA.iloc[0, 4]

(0.016500000000000001,
 0.018231000000000001,
 0.019226,
 0.019955000000000001,
 0.020569999999999998)

In [14]:
#####PART 2 of 2######

'''implementation of "forward_substitution" from https://en.wikipedia.org/wiki/Bootstrapping_(finance)
Namely extracting discount_factors from market traded yield_curve_instruments, which are futures and deposits up to 1year,
and interest (IRS) or overnight indexed swaps (OIS) from 1year onwards.  

Key concept on a fixed IRs: fair value is 0 at start, via no-arbitrage argument, assuming a notional of $1. Thus:

1year_IRS:  disc_factor1* ($1 + coup_1 * act/360) = $1   
2year_IRS:  disc_factor1 * coup_2 + disc_factor2*($1 + coup_2 * act/360) = $1
3year_IRS:  disc_factor1 * coup_3 + disc_factor2 * coup3 + disc_factor3 * ($1 + coup_3) = $1


Variation of Chapter5 from James WeiMing Ma "Mastering Python For Finance"   '''

import math

class GetDiscFactors(object):
   
    def __init__(self):
        
        self.instr = {}  #populated using add_instr() method
        self.disc_factors = {}
        self.zero_rates = {}
       
    def add_instr(self, par , T, coup, price , freq = 1): 
        
        #par and price only relevant for eurodollar futures, on IRS or OIS = $1 for both
       
        self.instr[T] = (par, coup, price, freq)
       
        '''dictionary with keys: T years grouping of tuples for vals:
            {0.25: (1, 0.0, 0.9849, 1),
             0.5: (1, 0.0, 0.9837, 1),
             1: (1, 0.013855, 1, 1),
             2: (1, 0.015271, 1, 1),
             3: (1, 0.01622, 1, 1)}
             and so forth'''
   
    def sort_instr(self):
       
        sort_dict = sorted(self.instr.keys())
        return sort_dict
    
    def get_df(self):
        #uses 2 helper methods below
        self.__get_shortdated_df__()
        self.__get_longdated_df__()
        return [self.disc_factors[i] for i in self.sort_instr()] #without this, get_df does not return anything.
        #you can still access disc_factors by calling rateObject.disc_factors
   
    def __get_shortdated_df__(self):
        
        '''Say eurodolly future price is 98.46 and there are 62days to next IMM date.
        thus yield is 100% - 98.46% = 1.54% p.a., thus disc_factor will be 1/(1+1.54%*62/360)  '''
        
        for T in self.instr:
           
            (par, coup, price, freq) = self.instr[T]  #without this: NameError: name 'coup' is not defined
            
            if coup == 0:
                self.disc_factors[T] = 1.0 / (1.0 + (1.0 - price)*T)
                #print(self.disc_factors.values())
    
    def __get_longdated_df__(self):
        #for T in self.instr:  #goes from 0.25, 0.50 straight to 2 skipping T = 1
        for T in self.sort_instr():
            
            #(par, coup, price, freq) = self.instr[T]
            instrument = self.instr[T]
            (par, coup, price, freq) = instrument  #without this; NameError: name 'coup' is not defined
            
            '''there is no need to 'bootstrap' for non coupon bearing instruments so by definition this procedure starts
            at T = 1yr. Year1 disc_factor is simply 1/1+ rate_y1 
            which we access in the dictionary, key = 1, and the 'coup' (ie [1]) column entry of the tuple '''
            
            self.disc_factors[1] = 1.0 / (1.0 + self.instr[1][1]*365.0/360.0)
            if coup != 0:
                for i in range(2, T+1):
                    sum_coup = 0
                    for k in range(1, i):
                        sum_coup += self.disc_factors[k] *self.instr[i][1] *365.0 / 360.0
                    self.disc_factors[i] = (1 - sum_coup) / (1 + self.instr[i][1]*365.0/360.0)
                    
            #return self.disc_factors
                    
    def get_zero_rates(self):
        
        #converts from discount factor to zero rate
        disc_factors = self.get_df() #returns a single list of dfs so use below instead
        disc_fact_dict = self.disc_factors #returns a dictionary
        zeroes = []
        for t, df in sorted(disc_fact_dict.items()):
            #zeroes.append(((1.0/df)**(1/t)) -1.0)
            self.zero_rates[t] = (((1.0/df)**(1/t)) -1.0)
        return(self.zero_rates) #[i] for i in self.zero_rates

                                  

In [15]:
curve = GetDiscFactors()
curve.add_instr(1, 0.25, 0., 0.9849)      # T = 0.25,   element 0
curve.add_instr(1, 0.50 , 0., 0.9837)     #  T = 0.50  element 1

#if yuo dont' have access to a data feed, just overwrite the "USSA.iloc[0,n] below with actual swap rates

curve.add_instr(1, 1, USSA.iloc[0,0], 1)  #Year 1   
curve.add_instr(1, 2, USSA.iloc[0,1], 1) #Year 2   
curve.add_instr(1, 3, USSA.iloc[0,2], 1) #Year 3  
curve.add_instr(1, 4, USSA.iloc[0,3], 1) #Year 4  
curve.add_instr(1, 5, USSA.iloc[0,4], 1) #Year 5  
curve.add_instr(1, 6, USSA.iloc[0,5], 1) #Year 6  
curve.add_instr(1, 7, USSA.iloc[0,6], 1) #Year 7  
curve.add_instr(1, 8, USSA.iloc[0,7], 1) #Year 8  
curve.add_instr(1, 9, USSA.iloc[0,8], 1) #Year 9  
curve.add_instr(1, 10, USSA.iloc[0,9], 1) #Year 10  
curve.add_instr(1, 11, USSA.iloc[0,10], 1) #Year 11 
curve.add_instr(1, 12, USSA.iloc[0,11], 1) #Year 12 
curve.add_instr(1, 15, USSA.iloc[0,12], 1) #Year 15
curve.add_instr(1, 20, USSA.iloc[0,13], 1) #Year 20 
curve.add_instr(1, 25, USSA.iloc[0,14], 1) #Year 25
curve.add_instr(1, 30, USSA.iloc[0,15], 1) #Year 30 '''


x = sorted(curve.instr.keys())
y = [curve.instr[i][1] for i in curve.instr]


us13 = np.interp(13, x, y)
curve.add_instr(1, 13, us13, 1) #Year 13   
us14 = np.interp(14, x, y)
curve.add_instr(1, 14, us14, 1) #Year 14 

us16 = np.interp(16, x, y)
curve.add_instr(1, 16, us16, 1) #Year 16 
us17 = np.interp(17, x, y)
curve.add_instr(1, 17, us14, 1) #Year 17 
us18 = np.interp(18, x, y)
curve.add_instr(1, 18, us18, 1) #Year 18 
us19 = np.interp(19, x, y)
curve.add_instr(1, 19, us19, 1) #Year 19

us21 = np.interp(21, x, y)
curve.add_instr(1, 21, us21, 1) #Year 21
us22 = np.interp(22, x, y)
curve.add_instr(1, 22, us22, 1) #Year 22
us23 = np.interp(23, x, y)
curve.add_instr(1, 23, us23, 1) #Year 23
us24 = np.interp(24, x, y)
curve.add_instr(1, 24, us24, 1) #Year 24
us26= np.interp(26, x, y)
curve.add_instr(1, 26, us26, 1) #Year 26
us27 = np.interp(27, x, y)
curve.add_instr(1, 27, us27, 1) #Year 27
us28 = np.interp(28, x, y)
curve.add_instr(1, 28, us28, 1) #Year 28
us29 = np.interp(29, x, y)
curve.add_instr(1, 29, us29, 1) #Year 29


curve.instr

{0.25: (1, 0.0, 0.9849, 1),
 0.5: (1, 0.0, 0.9837, 1),
 1: (1, 0.016500000000000001, 1, 1),
 2: (1, 0.018231000000000001, 1, 1),
 3: (1, 0.019226, 1, 1),
 4: (1, 0.019955000000000001, 1, 1),
 5: (1, 0.020569999999999998, 1, 1),
 6: (1, 0.02112, 1, 1),
 7: (1, 0.021673000000000001, 1, 1),
 8: (1, 0.022092000000000001, 1, 1),
 9: (1, 0.022505000000000001, 1, 1),
 10: (1, 0.022894999999999999, 1, 1),
 11: (1, 0.023285, 1, 1),
 12: (1, 0.023547999999999999, 1, 1),
 13: (1, 0.023949, 1, 1),
 14: (1, 0.023617, 1, 1),
 15: (1, 0.024281, 1, 1),
 16: (1, 0.0236006, 1, 1),
 17: (1, 0.023617, 1, 1),
 18: (1, 0.0242318, 1, 1),
 19: (1, 0.0245474, 1, 1),
 20: (1, 0.024863, 1, 1),
 21: (1, 0.0249088, 1, 1),
 22: (1, 0.0249546, 1, 1),
 23: (1, 0.0250004, 1, 1),
 24: (1, 0.0250462, 1, 1),
 25: (1, 0.025092, 1, 1),
 26: (1, 0.0251004, 1, 1),
 27: (1, 0.0251088, 1, 1),
 28: (1, 0.0251172, 1, 1),
 29: (1, 0.0251256, 1, 1),
 30: (1, 0.025134, 1, 1)}

In [18]:
#check output! below returns dictionary of disc_factor
#On 7November2017, BBG had 
#10yr df of 0.793193 vs 0.7934 
#15yr df of 0.69119 vs 0.6908
#30yr df of 0.464413 vs 0.46468
curve.get_df() #need this to fill up dictionary
curve.disc_factors

{0.25: 0.9962391970312071,
 0.5: 0.9919158855329067,
 1: 0.98354609347785993,
 2: 0.96400113135714449,
 3: 0.94364196873885375,
 4: 0.92283409806554861,
 5: 0.90165133097140793,
 6: 0.88017422337789508,
 7: 0.85817910026964195,
 8: 0.83669628310714705,
 9: 0.81504602433546625,
 10: 0.79342314718513929,
 11: 0.771685984260159,
 12: 0.7511729221601533,
 13: 0.72922877450193535,
 14: 0.71584158297071721,
 15: 0.69084498726625265,
 16: 0.68316115730295746,
 17: 0.66697037193610142,
 18: 0.64251536860485492,
 19: 0.62236966895287282,
 20: 0.60233081918467279,
 21: 0.58677930694090341,
 22: 0.57155840612775688,
 23: 0.55666201167707063,
 24: 0.54208411093123876,
 25: 0.52781878271195615,
 26: 0.51456552490550356,
 27: 0.50163273915722295,
 28: 0.4890127892076977,
 29: 0.47669821810290947,
 30: 0.46468174404818285}

In [20]:
#check output! below returns dictionary of zero_rates
#On 7November2017, BBG had: 
#10yr zero of 2.33036 vs 2.3409
#15yr zero of 2.47743 vs 2.4963
#30yr zero of 2.57229 vs 2.5875

curve.get_zero_rates()
curve.zero_rates

{0.25: 0.015185719137518205,
 0.5: 0.0163664225000002,
 1: 0.016729166666666684,
 2: 0.018500456998932435,
 3: 0.019524305878110715,
 4: 0.02027933809159399,
 5: 0.020921322477683413,
 6: 0.021500442367663908,
 7: 0.022089358422466621,
 8: 0.022536972547602785,
 9: 0.022983554046954513,
 10: 0.023409663271541481,
 11: 0.023841364439493251,
 12: 0.0241298069205369,
 13: 0.02458722992132012,
 14: 0.024165683194747212,
 15: 0.024962459894586564,
 16: 0.024099849111722804,
 17: 0.024110158294165851,
 18: 0.024880282200891557,
 19: 0.025273085608944923,
 20: 0.025671400008550282,
 21: 0.025710992858211323,
 22: 0.025752771931649043,
 23: 0.025796553190745808,
 24: 0.025842186892884422,
 25: 0.025889550890745072,
 26: 0.02588442208104369,
 27: 0.025880605632351728,
 28: 0.025877976542292735,
 29: 0.025876427477690811,
 30: 0.025875865841861856}

In [207]:
#plot discount curve
x = curve.sort_instr()
y = curve.get_df()
plt.plot(x,y)
plt.title("Discount Factors from IRS")
plt.xlabel("Tenor")
plt.ylabel("Discount Factors")
plt.grid('on')

#set limits
plt.xlim(0,31)
plt.ylim(0, 1.2)

plt.show()

In [36]:
'''scrap paper'''
#0.016031,  Y1
#0.017877999999999998,  Y2
#0.019050999999999998,  Y3
#0.019889999999999998,   Y4
#0.020573000000000001)' Y5

df1 = (1 - 0.0)/ (1 + USSA.iloc[0,0]*365/360)

df2 = (1 - df1 * USSA.iloc[0, 1]*365/360)  / (1 + USSA.iloc[0,1]*365/360)

df3 = (1 - ((df1 + df2)* USSA.iloc[0, 2]*365/360))  / (1 + USSA.iloc[0,2]*365/360)

df4 = (1 - ((df1 + df2 + df3) * USSA.iloc[0, 3]*365/360)) / (1 + USSA.iloc[0,3]*365/360)
       
df5 = (1 - ((df1 + df2 + df3 + df4)* USSA.iloc[0, 4]*365/360)) / (1 + USSA.iloc[0,4]*365/360)

df_tuple = df1, df2, df3, df4, df5

year = 1

for i in df_tuple:
    
    print('year '+str(year)+ ' '  + str(i) )
    year += 1


sumcoup2 = (df1) * 0.017877999999999998
sumcoup3 = (df1 + df2 ) * 0.019050999999999998
sumcoup4 = (df1 + df2 + df3)* 0.019889999999999998
sumcoup2 , sumcoup3, sumcoup4


[0, 1, 2, 3]


[0.9842219381101561,
 0.9646737217730607,
 0.9441196809193477,
 0.9230443855435724,
 0.9015956710035242]

In [ ]:
#THIS WORKS PERFECTLY
def get_df_list(n):
    
    '''
    (rate_n)*df1 + (rate_n)*df2 ...+ (1+ rate_n)*df_n = $1 
    Thus  df_n =  ($1 - sum(from 1 to n-1) *df[1 to n-1]  * r[n] )  / (1 + rate_n)'''
    
    rates = [0.016031, 0.017877999999999998, 0.019050999999999998,0.019889999999999998, 0.0205730000000000]
    
    df = [] #initiailze
   
    #populate 1st year discount factor 
    df.insert(0, 1/(1+rates[0]))
    
    for i in range(1, len(rates)):
        sum_df = 0
        j_vec = []
        #for j in range(0, len(rates)-1): #IndexError: list index out of range
        for j in range(0, i):
            j_vec.append(j)
            sum_df +=  df[j] * rates[i]*(365/360)    
        df.append((1-sum_df)/(1+rates[i]*365/360))
    print(j_vec)
         
    return(df)
        
get_df_list(3)

In [37]:
def get_df_dictio(n):
    
    rates_keys = [1, 2, 3, 4, 5]
    rates_val = [0.016031, 0.017877999999999998, 0.019050999999999998,0.019889999999999998, 0.0205730000000000]
    zipped = list(zip(rates_keys, rates_val))
    rates = dict(zipped)
    (coup) = rates
    
    #populate 1st year discount factor 
    df = {}
    df[1] = 1/(1+rates[1])
    
    for i in range(2, len(rates)+1):
        sum_df = 0
        #print('i '+str(n))
        
        for j in range(1, i):
            #print(j)
            #print('df[j] '+str(df[j]))
            sum_df +=  df[j] * rates[i]*(365/360)    
        df[i] = ((1-sum_df)/(1+rates[i]*365/360))
        
    return(df)
        
get_df_dictio(6)

{1: 0.9842219381101561,
 2: 0.9646737217730607,
 3: 0.9441196809193477,
 4: 0.9230443855435724,
 5: 0.9015956710035242}

In [51]:
dictio = {}
def add_instr2(par, T, coup, price, freq = 1):
    dictio[T] = (par, coup, price, freq)
    
add_instr2(1, 1, 0.016031, 1)  #Year 1   
add_instr2(1, 2, 0.017877999999999998, 1) #Year 2   
add_instr2(1, 3, 0.019050999999999998, 1) #Year 3  
add_instr2(1, 4, 0.019889999999999998, 1) #Year 4
dictio


{1: (1, 0.016031, 1, 1),
 2: (1, 0.017877999999999998, 1, 1),
 3: (1, 0.019051, 1, 1),
 4: (1, 0.019889999999999998, 1, 1)}

In [56]:
dictio[2][1]

0.017877999999999998

In [59]:
for i in range(1,len(dictio)):
    print(dictio[i][2])

1
1
1


In [91]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np

In [97]:
x = np.arange(0, 10)
x
f = interpolate.interp1d(x, y)

In [94]:
y = np.exp(x)
y

array([  1.00000000e+00,   2.71828183e+00,   7.38905610e+00,
         2.00855369e+01,   5.45981500e+01,   1.48413159e+02,
         4.03428793e+02,   1.09663316e+03,   2.98095799e+03,
         8.10308393e+03])

In [99]:
xnew = np.arange(0, 9, 0.1)
xnew
ynew = f(xnew)
ynew
plt.plot(x, y, 'o', xnew, ynew, '-')

In [100]:
plt.show()